In [17]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
import os
import math
import copy

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

prev_right_lines = []
prev_left_lines = []

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
    
    for v in vertices:
    #filling pixels inside the polygon defined by "vertices" with the fill color    
        cv2.fillPoly(mask, v, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

# returns a new list with lines that have a min angle of 0.3
def filter_lines(lines):
    filtered_lines = []
    for line in lines:
        angle = (abs(line[0][3]-line[0][1]))/(abs(line[0][2]-line[0][0]))
        if angle > 0.3:
            filtered_lines.append(line)
            
    return filtered_lines

def avarage_lines (lines):
    right_lines = []
    left_lines = []
    all_lines = [] # for debugging only
    tmp = []
    adjust_lines = {'right' : None, 'left' : None}
    left_line = None
    right_line = None
    result = []
    
    lines = filter_lines(lines)
    
    
    
    # starting values for top points
#     y_top_right = imshape[0]
#     y_top_left = imshape[0]
#     x_top_right = 0
#     x_top_left = 0
    
    # sort lines in left and right
    for line in lines:
        for x1,y1,x2,y2 in line:
            angle = (y2-y1)/(x2-x1)
            if(angle > 0): # positive angle means right side
                right_lines.append(line)
            else:
                left_lines.append(line)

    # add lines from previuos image
    global prev_right_lines
    
    if len(prev_right_lines) == 0:
        for line in right_lines:
            prev_right_lines.append(line)
    else:
        tmp = prev_right_lines[:]
        prev_right_lines = right_lines[:]
        for line in tmp:
            right_lines.append(line)
    
    tmp = []
    
    global prev_left_lines
    
    if len(prev_left_lines) == 0:
        for line in left_lines:
            prev_left_lines.append(line)
    else:
        tmp = prev_left_lines[:]
        prev_left_lines = left_lines[:]
        for line in tmp:
            left_lines.append(line)
    
    # numbers needed to calculate line angles
    x1_tot_right = 0
    x2_tot_right = 0
    y1_tot_right = 0
    y2_tot_right = 0
    
    y1_tot_left = 0
    y2_tot_left = 0
    x1_tot_left = 0
    x2_tot_left = 0 
    
    for line in right_lines:
        for x1,y1,x2,y2 in line:
            x1_tot_right += x1
            x2_tot_right += x2
            y1_tot_right += y1
            y2_tot_right += y2
                
    for line in left_lines:
        for x1,y1,x2,y2 in line:
            x1_tot_left += x1
            x2_tot_left += x2
            y1_tot_left += y1
            y2_tot_left += y2
            
    # top border of the mask pollygraphs
    pY_top = int(imshape[0]*0.6)
    pY_bottom = imshape[0]
    
    #Extend all lines with avarage angle to find the avarage X starting point and make a line from it.
    #First the left...
    if len(left_lines) > 0:
        av_angle_left = (y2_tot_left-y1_tot_left)/(x2_tot_left-x1_tot_left)
        x_tot = 0
        for line in left_lines:
            x_top_left = line[0][0]
            y_top_left = line[0][1]
            x_tot += x_top_left-int(((pY_bottom-y_top_left)/abs(av_angle_left)))
            
        av_x = int(x_tot / len(left_lines))
        left_line = [[av_x+int(((pY_bottom - pY_top)/abs(av_angle_left))), pY_top, av_x, pY_bottom]]

    if left_line is not None:
        result.append(left_line)
    adjust_lines['left'] = left_line
    
    #...and then the right.
    if len(right_lines) > 0:
        av_angle_right = (y2_tot_right-y1_tot_right)/(x2_tot_right-x1_tot_right)
        x_tot = 0
        for line in right_lines:
            x_top_right = line[0][0]
            y_top_right = line[0][1]
            x_tot += x_top_right+int(((pY_bottom-y_top_right)/abs(av_angle_right)))
            
        av_x = int(x_tot / len(right_lines))
        right_line = [[av_x-int(((pY_bottom - pY_top)/abs(av_angle_right))), pY_top, av_x, pY_bottom]]
    
    
    if right_line is not None:
        result.append(right_line)
    adjust_lines['right'] = right_line
    
    #Adjust the mask using the right and left lines.
    #TODO: adjust_lines are the same as in result. Have not yet found a way to use result in the adjust_masks() function
            
    adjust_masks(adjust_lines)

# two lines useful for debugging
#     all_lines = right_lines + left_lines
#     return all_lines
    return result


def adjust_masks(lines):
    # probably a lot of syntax to improve here due to lacking python skills
    ll = np.array(lines['left'])
    rl = np.array(lines['right'])
    
    rp = mask_pollys[0][0]
    lp = mask_pollys[1][0]
    
    pY_top = int(imshape[0]*0.6)
    pY_bottom = imshape[0]
    
    if lines['right'] is not None:
        rp[0][0] = rl[0][2]-50
        rp[3][0] = rl[0][2]+50
        rp[1][0] = rl[0][0]-30
        rp[2][0] = rl[0][0]+30
        rp[1][1] = pY_top
        rp[2][1] = pY_top
    else:
        mask_pollys[0] = np.copy(START_MASK_RIGHT)
        
    if lines['left'] is not None:
        lp[0][0] = ll[0][2]-50
        lp[3][0] = ll[0][2]+50
        lp[1][0] = ll[0][0]-30
        lp[2][0] = ll[0][0]+30
        lp[1][1] = pY_top
        lp[2][1] = pY_top
    else:
        mask_pollys[1] = np.copy(START_MASK_LEFT)
    

def draw_lines(img, lines, color=[255, 0, 0], thickness=6):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    lines = avarage_lines(lines)
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def add_blue_line(image, l, color=[0, 0, 255], thickness=6):
    p1 = l[0]
    p2 = l[1]
    cv2.line(image, (p1[0], p1[1]), (p2[0], p2[1]), color, thickness)

def add_polly_of_interest(image, v):
        bl = v[0][0]
        tl = v[0][1]
        tr = v[0][2]
        br = v[0][3]
        lines = [[bl, tl], [tl, tr], [tr, br], [br, bl]]
        for l in lines:
            add_blue_line(image, l)
        
def add_region_of_interest_lines(image, masks):
    
        
    for vertices in masks:
        add_polly_of_interest(image, vertices)

# *************************************************************
# Here are all parameters for gaussian, canny, hough and region of interest starting points.
# Starting points for mask region is global so it can be reset if lines is lost.
        
#region_of_interest
mask_pollys = []
imshape = None
START_MASK_RIGHT = None
START_MASK_LEFT = None

def process_image(image):

    #gaussianblur
    kernel_size = 5
    
    #canny
    low_threshold = 210
    high_threshold = 230
    
    #Hough
    rho = 2
    theta = 1 * np.pi/180
    threshold = 30
    min_line_len = 20
    max_line_gap = 20
    TEST_max_line_gap = 50
    
#region_of_interest
    global imshape
    global START_MASK_RIGHT, START_MASK_LEFT
    imshape = image.shape
    iH = imshape[0]
    iW = imshape[1]
    if len(mask_pollys) == 0:
        START_MASK_RIGHT = np.array([[(iW*0.7, iH),(iW*0.50, iH*0.65), (iW*0.60, iH*0.65), (iW , iH)]], dtype=np.int32)
        START_MASK_LEFT = np.array([[(0, iH),(iW*0.40, iH*0.65), (iW*0.50, iH*0.65), (iW*0.3 , iH)]], dtype=np.int32)
        mask_pollys.append(np.copy(START_MASK_RIGHT))
        mask_pollys.append(np.copy(START_MASK_LEFT))
    
    initial_image = image
    blur_image = gaussian_blur(initial_image, kernel_size)
    gray_image = grayscale(blur_image)
    canny_image = canny(gray_image, low_threshold, high_threshold)
    cut_image = region_of_interest(canny_image, mask_pollys)
    line_image = hough_lines(cut_image, rho, theta, threshold, min_line_len, max_line_gap)
    w_image = weighted_img(line_image, initial_image, α=0.8, β=1., λ=0.)
    add_region_of_interest_lines(w_image, mask_pollys)
    image = w_image
    
# for debugging
#     plt.imshow(initial_image)
#     plt.show()
#     plt.imshow(blur_image)
#     plt.show()
#     plt.imshow(gray_image, cmap='gray')
#     plt.show()
#     plt.imshow(canny_image)
#     plt.show()
#     plt.imshow(cut_image)
#     plt.show()
#     plt.imshow(line_image)
#     plt.show()
#     plt.imshow(w_image)
    
#     plt.show()
    
    
    return image



# white_output = 'white.mp4'
# clip1 = VideoFileClip("solidWhiteRight.mp4")
# white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
# %time white_clip.write_videofile(white_output, audio=False)

# yellow_output = 'yellow.mp4'
# clip2 = VideoFileClip('solidYellowLeft.mp4')
# yellow_clip = clip2.fl_image(process_image)
# %time yellow_clip.write_videofile(yellow_output, audio=False)

challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 251/251 [00:04<00:00, 50.22it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 5.58 s
